<a href="https://colab.research.google.com/github/OdysseusPolymetis/ia_et_shs/blob/main/2_postagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers nltk

In [5]:
from transformers import CamembertTokenizer, CamembertForTokenClassification, TokenClassificationPipeline

tokenizer = CamembertTokenizer.from_pretrained('qanastek/pos-french-camembert')
model = CamembertForTokenClassification.from_pretrained('qanastek/pos-french-camembert')
pos = TokenClassificationPipeline(model=model, tokenizer=tokenizer)

In [6]:
def make_prediction(sentence):
    labels = [l['entity'] for l in pos(sentence)]
    return list(zip(sentence.split(" "), labels))

In [14]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [12]:
text = """
Tant qu’il existera, par le fait des lois et des mœurs, une damnation sociale créant artificiellement, en pleine civilisation, des enfers, et compliquant d’une fatalité humaine la destinée qui est divine ; tant que les trois problèmes du siècle, la dégradation de l’homme par le prolétariat, la déchéance de la femme par la faim, l’atrophie de l’enfant par la nuit, ne seront pas résolus ; tant que, dans de certaines régions, l’asphyxie sociale sera possible ; en d’autres termes, et à un point de vue plus étendu encore, tant qu’il y aura sur la terre ignorance et misère, des livres de la nature de celui-ci pourront ne pas être inutiles.
"""

In [19]:
text = "Aujourd'hui, maman est morte. Ou peut-être hier, je ne sais pas. J'ai reçu un télégramme de l'asile : “Mère décédée. Enterrement demain. Sentiments distingués.” Cela ne veut rien dire. C'était peut-être hier."

In [20]:
sentences = sent_tokenize(text)

In [21]:
len(sentences)

6

In [22]:
for i, sentence in enumerate(sentences):
    print(f"\nPhrase {i+1}: {sentence}")
    predictions = make_prediction(sentence)
    print("Analyse POS :")
    for word, tag in predictions:
        print(f"  {word} -> {tag}")


Phrase 1: Aujourd'hui, maman est morte.
Analyse POS :
  Aujourd'hui, -> ADV
  maman -> ADV
  est -> ADV
  morte. -> PUNCT

Phrase 2: Ou peut-être hier, je ne sais pas.
Analyse POS :
  Ou -> COCO
  peut-être -> ADV
  hier, -> ADV
  je -> ADV
  ne -> ADV
  sais -> PUNCT
  pas. -> PPER1S

Phrase 3: J'ai reçu un télégramme de l'asile : “Mère décédée.
Analyse POS :
  J'ai -> PPER1S
  reçu -> ADV
  un -> AUX
  télégramme -> VPPMS
  de -> DINTMS
  l'asile -> NMS
  : -> NMS
  “Mère -> PREP
  décédée. -> DET

Phrase 4: Enterrement demain.
Analyse POS :
  Enterrement -> NMS
  demain. -> NMS

Phrase 5: Sentiments distingués.” Cela ne veut rien dire.
Analyse POS :
  Sentiments -> NMP
  distingués.” -> NMP
  Cela -> NMP
  ne -> ADJMP
  veut -> ADJMP
  rien -> PUNCT
  dire. -> PDEMMS

Phrase 6: C'était peut-être hier.
Analyse POS :
  C'était -> PDEMMS
  peut-être -> ADV
  hier. -> AUX


In [27]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("gilf/french-camembert-postag-model")
model = AutoModelForTokenClassification.from_pretrained("gilf/french-camembert-postag-model")

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at gilf/french-camembert-postag-model were not used when initializing CamembertForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
from transformers import pipeline

nlp_token_class = pipeline('ner', model=model, tokenizer=tokenizer, grouped_entities=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [31]:
for i, sentence in enumerate(sentences):
    print(f"\nPhrase {i+1}: {sentence}")
    results = nlp_token_class(sentence)
    print("Analyse POS :")
    for result in results:
        print(f"  {result['word']} -> {result['entity_group']}")


Phrase 1: Aujourd'hui, maman est morte.
Analyse POS :
  Aujourd -> ADV
  ' -> NC
  hui -> ADV
  , -> P
  maman -> NC
  est -> V
  morte -> ADJ
  . -> NC

Phrase 2: Ou peut-être hier, je ne sais pas.
Analyse POS :
  Ou -> CC
  peut-être hier -> ADV
  , -> PONCT
  je -> CLS
  ne -> ADV
  sais -> V
  pas -> ADV
  . -> NC

Phrase 3: J'ai reçu un télégramme de l'asile : “Mère décédée.
Analyse POS :
  J -> CLS
  ' -> NC
  ai -> V
  reçu -> VPP
  un -> DET
  télégramme -> NC
  de -> P
  l -> DET
  'asile -> NC
  : -> PONCT
  “Mère -> NC
  décédée -> VPP
  . -> NC

Phrase 4: Enterrement demain.
Analyse POS :
  En -> ADV
  terre -> NC
  ment demain -> ADV
  . -> NC

Phrase 5: Sentiments distingués.” Cela ne veut rien dire.
Analyse POS :
  Sen -> U
  ti -> ADJ
  ments -> NC
  distingués -> VPP
  .” -> NC
  Cela -> PRO
  ne -> ADV
  veut -> V
  rien -> PRO
  dire -> VINF
  . -> NC

Phrase 6: C'était peut-être hier.
Analyse POS :
  C -> CLS
  ' -> NC
  était -> V
  peut-être hier -> ADV
  . -> NC